In [17]:
import pandas as pd

df = pd.read_csv('analyst_ratings_processed.csv')

In [18]:
print(df.columns)
print(df.head(5))
print(df.shape)

Index(['Unnamed: 0', 'title', 'date', 'stock'], dtype='object')
   Unnamed: 0                                              title  \
0         0.0            Stocks That Hit 52-Week Highs On Friday   
1         1.0         Stocks That Hit 52-Week Highs On Wednesday   
2         2.0                      71 Biggest Movers From Friday   
3         3.0       46 Stocks Moving In Friday's Mid-Day Session   
4         4.0  B of A Securities Maintains Neutral on Agilent...   

                        date stock  
0  2020-06-05 10:30:00-04:00     A  
1  2020-06-03 10:45:00-04:00     A  
2  2020-05-26 04:30:00-04:00     A  
3  2020-05-22 12:45:00-04:00     A  
4  2020-05-22 11:38:00-04:00     A  
(1400469, 4)


In [ ]:
import pandas as pd

# S&P500 ticker list from Wikipedia
# sp500_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
# sp500_tickers = sp500_table['Symbol'].tolist()
df_snp = pd.read_csv("all_stocks_5yr.csv")
df_snp['date'] = pd.to_datetime(df_snp['date'])
df_snp.set_index('date', inplace=True)
df_snp = df_snp.iloc[::-1]
snp500_tickers = df_snp['Name'].unique()
snp500_tickers = snp500_tickers.tolist()

# Filter your DataFrame (assumes your tickers are in df['Name'] or df['ticker'])
filtered_df = df[df['stock'].isin(sp500_tickers)]

# Optional: check how many rows were kept
print(f"Filtered rows: {len(filtered_df)}")

Filtered rows: 333509


**Installing finBERT**

In [21]:
!pip install transformers --quiet

In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


Device set to use cuda:0


In [29]:
print(len(filtered_df))
results = sentiment_pipeline(filtered_df['title'].tolist(), batch_size=32)

filtered_df['sentiment'] = [r['label'].lower() for r in results]

print(filtered_df.head(5))

filtered_df.to_csv('finBert_sentiment.csv', index=False)

333509


<ipython-input-29-646f4f10a21f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sentiment'] = [r['label'].lower() for r in results]


   Unnamed: 0                                              title  \
0         0.0            Stocks That Hit 52-Week Highs On Friday   
1         1.0         Stocks That Hit 52-Week Highs On Wednesday   
2         2.0                      71 Biggest Movers From Friday   
3         3.0       46 Stocks Moving In Friday's Mid-Day Session   
4         4.0  B of A Securities Maintains Neutral on Agilent...   

                        date stock sentiment  
0  2020-06-05 10:30:00-04:00     A   neutral  
1  2020-06-03 10:45:00-04:00     A   neutral  
2  2020-05-26 04:30:00-04:00     A   neutral  
3  2020-05-22 12:45:00-04:00     A   neutral  
4  2020-05-22 11:38:00-04:00     A  positive  


In [33]:
import pandas as pd

df1 = pd.read_csv('finBert_sentiment.csv')

total_rows = len(df1)

sentiment_counts = df1['sentiment'].value_counts()

# Display results
print(f"Total rows: {total_rows}")
print("Sentiment counts:")
print(sentiment_counts)


Total rows: 333509
Sentiment counts:
sentiment
neutral     195700
positive     86216
negative     51593
Name: count, dtype: int64
